In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Add the src directory to the Python path

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.Template import Template
from src.utils import download_data
from src.mappings import ideology_mapping

In [2]:
template = Template(categories = ['sc'], file_path= '../datasets/test.csv')


Loaded template from ../datasets/test.csv


In [3]:
df = pd.read_csv('../datasets/Equinox2020.05.2023_MSP.csv', sep = '|')
polity_df = download_data("https://seshatdata.com/api/core/polities/")

Downloaded 839 rows


In [17]:
df = pd.read_csv('../datasets/mr_dataset.04.2021.csv')
polity_df = download_data("https://seshatdata.com/api/core/polities/")

Downloaded 839 rows


In [18]:
df

,NGA,PolID,Start,End,Dupl,primary,certain,broad,targeted,ruler,...,commoners,afterlife,thislife,agency,disputed,expert,infA,MSP,MSP_sum,minMSP
0,Latium,ItLatCA,-3600,-1801,n,0.0,0.0,0.0,1.0,1.0,...,1.0,NaN,1.0,1.0,no,checked,1,0.125,4.0,1.0
1,Latium,ItLatBA,-1800,-1001,n,0.0,0.0,0.0,1.0,1.0,...,1.0,NaN,1.0,1.0,no,checked,1,0.125,4.0,1.0
2,Latium,ItLatIA,-1000,-717,n,0.0,0.0,0.0,1.0,1.0,...,1.0,NaN,1.0,1.0,no,checked,1,0.125,4.0,1.0
3,Latium,ItRomRg,-716,-510,n,0.0,0.0,0.0,1.0,1.0,...,1.0,NaN,1.0,1.0,no,checked,1,0.125,4.0,1.0
4,Latium,ItRomER,-509,-265,n,0.0,0.0,0.0,1.0,1.0,...,1.0,NaN,1.0,1.0,no,checked,0,0.125,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,Middle Ganga,InGupta,320,514,n,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,no,checked,0,1.000,7.0,1.0
344,Middle Ganga,InMagad,550,606,n,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,no,checked,0,1.000,7.0,1.0
345,Middle Ganga,InKanau,650,779,n,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,no,checked,0,1.000,7.0,1.0
346,Middle Ganga,InGurjr,810,1030,n,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,no,checked,0,1.000,7.0,1.0


In [5]:
column_mappings = {}
for column in df.columns:
    column_mappings[column] = column.lower().replace('.', '_').replace('-', '_')
column_mappings['Date.From'] = 'year_from'
column_mappings['Date.To'] = 'year_to'
column_mappings['Variable'] = 'name'
df.rename(columns=column_mappings, inplace=True)

In [6]:
df.apply(lambda x: int(polity_df.loc[polity_df['name'] == x['polid'], 'id'].values[0]) if x['polid'] in polity_df['name'].values else print(x.polid), axis=1);

GbEmpir
AkKidar
InEInCo
InBritP
InKanau
InGurjr


In [7]:
df['polity_id'] = df.apply(lambda x: int(polity_df.loc[polity_df['name'] == x['polid'], 'id'].values[0]) if x['polid'] in polity_df['name'].values else None, axis=1)
print(df.polity_id.isna().sum())
df = df[df.polity_id.notna()]
df.polity_id = df.polity_id.astype(int)

df['polity_new_name'] = df.apply(lambda x: polity_df.loc[polity_df['id'] == x['polity_id'], 'new_name'].values[0] if x['polity_id'] in polity_df['id'].values else None, axis=1)

6


In [8]:
print(df.columns)

Index(['nga', 'polid', 'start', 'end', 'dupl', 'primary', 'certain', 'broad',
       'targeted', 'ruler', 'elite', 'commoners', 'afterlife', 'thislife',
       'agency', 'disputed', 'expert', 'infa', 'msp', 'msp_sum', 'minmsp',
       'polity_id', 'polity_new_name'],
      dtype='object')


In [9]:
ideology_mapping['MSP']

{'moral-concern-is-primary': 1,
 'moralizing-enforcement-is-certain': 1,
 'moralizing-norms-are-broad': 1,
 'moralizing-enforcement-is-targeted': 1,
 'moralizing-enforcement-of-rulers': 1,
 'moralizing-religion-adopted-by-elites': 1,
 'moralizing-religion-adopted-by-commoners': 1,
 'moralizing-enforcement-in-afterlife': 1,
 'moralizing-enforcement-in-this-life': 1,
 'moralizing-enforcement-is-agentic': 1}

In [10]:
col_replace = {'primary' : 'moral-concern-is-primary',
               'certain' : 'moralizing-enforcement-is-certain',
               'broad' : 'moralizing-norms-are-broad',
               'targeted' : 'moralizing-enforcement-is-targeted',
               'ruler': 'moralizing-enforcement-of-rulers',
               'elite' : 'moralizing-religion-adopted-by-elites',
               'commoners' : 'moralizing-religion-adopted-by-commoners',
               'afterlife' : 'moralizing-enforcement-in-afterlife',
               'thislife' : 'moralizing-enforcement-in-this-life',
               'agency' : 'moralizing-enforcement-is-agentic',
               'start' : 'year_from',
                'end' : 'year_to',
               }

df.rename(columns=col_replace, inplace=True)

In [11]:
df['is_disputed'] = False
df['is_uncertain'] = False

In [12]:
variables = [col for col in col_replace.values()]

In [13]:

for variable in variables:

    if (variable in ideology_mapping['MSP'].keys()):
        print(variable)
        var_df = df.copy()
        var_df['name'] = variable
        # make sure that if an element of the col variable has more than one entry, the first one is taken
        template.add_to_template(var_df, variable)

moral-concern-is-primary
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the 

In [15]:
pols = template.template.PolityName.unique()
msp_pols = df.polity_new_name.unique()

# find polities that are in both datasets
common_pols = set(pols).intersection(set(msp_pols))
print(f"found {len(common_pols)} common polities")
# find polities only in pols
pols_only = set(pols).difference(set(msp_pols))
print(f"found {len(pols_only)} polities only in pols")

# find polities only in pt_pols
msp_pols_only = set(msp_pols).difference(set(pols))
print(f"found {len(msp_pols_only)} polities only in msp_pols")
print(msp_pols_only)

found 337 common polities
found 526 polities only in pols
found 2 polities only in msp_pols
{'ir_ak_koynlu', 'ye_resulid_dyn'}


In [16]:
template.save_dataset("../datasets/MSP_template_new.csv")

Saved template to ../datasets/MSP_template_new.csv


In [27]:
df['is_disputed'] = df.disputed == 'yes'
df['is_uncertain'] = df.value_note.apply(lambda x: True if x == 'uncertain' else False)

AttributeError: 'DataFrame' object has no attribute 'value_note'

In [7]:
variables = df.name.unique()

for variable in variables:
    var_df = df.loc[df['name'] == variable]
    variable = variable.replace(' ', '-').lower()
    if (variable in ideology_mapping['MSP'].keys()):
        print(variable)
        variable = variable.lower()
        var_df.name = variable
        var_df.rename(columns={'value_from': variable}, inplace=True)
        template.add_to_template(var_df, variable)

moral-concern-is-primary


/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_27835/4136551396.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.name = variable
/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_27835/4136551396.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.rename(columns={'value_from': variable}, inplace=True)


Added moral-concern-is-primary dataset to template
moralizing-enforcement-is-certain


/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_27835/4136551396.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.name = variable
/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_27835/4136551396.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.rename(columns={'value_from': variable}, inplace=True)


Added moralizing-enforcement-is-certain dataset to template
moralizing-norms-are-broad
Added moralizing-norms-are-broad dataset to template
moralizing-enforcement-is-targeted


...

Added moralizing-enforcement-in-afterlife dataset to template
moralizing-enforcement-in-this-life


/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_27835/4136551396.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.name = variable
/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_27835/4136551396.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.rename(columns={'value_from': variable}, inplace=True)


Added moralizing-enforcement-in-this-life dataset to template
moralizing-enforcement-is-agentic


/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_27835/4136551396.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.name = variable
/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_27835/4136551396.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.rename(columns={'value_from': variable}, inplace=True)


Added moralizing-enforcement-is-agentic dataset to template


In [8]:
template.save_dataset("../datasets/MSP_template.csv")

Saved template to ../datasets/MSP_template.csv
